## Calling libc

Or anything else that's linked into your process memory.

In [1]:
ccall(:puts, Cint, (Cstring,), "Hello")

Hello


6

In [2]:
@time ccall(:sleep, Cuint, (Cuint,), 3)

  3.000139 seconds


0x00000000

## Calling shared libraries

Let's compile a little C program as a shared library:

In [3]:
;cat hello.c

// Linux: gcc -shared -o libhello.so hello.c
// macOS: gcc -shared -o libhello.dylib hello.c
// Windows: compiler? what compiler?

#include <stdio.h>

void hello(char *name) {
    printf("Hello, %s!\n", name);
}

double sqr(double x) {
    return x*x;
}


In [4]:
;gcc -shared -fPIC -o libhello.so hello.c

In [5]:
ccall((:hello, "./libhello.so"), Cvoid, (Cstring,), "ORNL")

Hello, ORNL!


In [6]:
ccall((:hello, "./libhello.so"), Cvoid, (Cstring,), "Greg")

Hello, Greg!


In [7]:
floatmax()

1.7976931348623157e308

In [14]:
ccall((:sqr, "./libhello.so"), Cdouble, (Cdouble,), 2.0)

4.0

## Callbacks: calling Julia from C

In [15]:
T = Float64

# C-friendly callback function
function callback(p_a::Ptr{T}, p_b::Ptr{T})::Cint
    a = unsafe_load(p_a)
    b = unsafe_load(p_b)
    a < b ? -1 : b < a ? 1 : 0
end

callback (generic function with 1 method)

In [16]:
p = [1.2, 3.4]

2-element Array{Float64,1}:
 1.2
 3.4

In [17]:
p_a = pointer(p, 1)
p_b = pointer(p, 2)
callback(p_b, p_a)

1

In [ ]:
GC.@preserve a b c begin

end


In [18]:
# get C-callable function pointer
p_callback = @cfunction(callback, Cint, (Ptr{T}, Ptr{T}))

Ptr{Nothing} @0x00007fc668e16b60

In [19]:
A = randn(10)

10-element Array{Float64,1}:
 -0.25223521226712214
  0.5283980051823385 
  0.617945306851973  
 -0.32320845351905403
  0.49911870449233414
 -0.5822082084777798 
  0.12836060626513401
 -0.24160031841186516
  0.5712914028627493 
 -1.3265522378215313 

In [20]:
# call C's qsort function
ccall(:qsort, Cvoid,
    (Ptr{T}, Csize_t, Csize_t, Ptr{Cvoid}),
    A, length(A), sizeof(T), p_callback)

In [21]:
A

10-element Array{Float64,1}:
 -1.3265522378215313 
 -0.5822082084777798 
 -0.32320845351905403
 -0.25223521226712214
 -0.24160031841186516
  0.12836060626513401
  0.49911870449233414
  0.5283980051823385 
  0.5712914028627493 
  0.617945306851973  

In [22]:
function qsort!((<)::Function, A::Vector{T}) where T

    # C-friendly callback function
    function callback(p_a::Ptr{T}, p_b::Ptr{T})::Cint
        a = unsafe_load(p_a)
        b = unsafe_load(p_b)
        a < b ? -1 : b < a ? 1 : 0
    end

    # get C-callable function pointer
    p_callback = @cfunction($callback, Cint, (Ptr{T}, Ptr{T}))

    # call C's qsort function
    ccall(:qsort, Cvoid,
        (Ptr{T}, Csize_t, Csize_t, Ptr{Cvoid}),
        A, length(A), sizeof(T), p_callback)
    
    return A
end

# default comparison by `isless` function
qsort!(A::Vector) = qsort!(isless, A)

qsort! (generic function with 2 methods)

In [23]:
A = randn(10)

10-element Array{Float64,1}:
 -0.3004077766954796 
  0.4000188338534922 
  0.02030449583778625
  0.5849497180762366 
 -1.81397292452982   
 -1.0761369195107533 
 -0.0504877330046909 
 -0.8167934875972962 
  1.7551050989528332 
  0.06947041068293412

In [24]:
qsort!(A)

10-element Array{Float64,1}:
 -1.81397292452982   
 -1.0761369195107533 
 -0.8167934875972962 
 -0.3004077766954796 
 -0.0504877330046909 
  0.02030449583778625
  0.06947041068293412
  0.4000188338534922 
  0.5849497180762366 
  1.7551050989528332 

In [26]:
qsort!((x,y)->abs(y) < abs(x), A)

10-element Array{Float64,1}:
 -1.81397292452982   
  1.7551050989528332 
 -1.0761369195107533 
 -0.8167934875972962 
  0.5849497180762366 
  0.4000188338534922 
 -0.3004077766954796 
  0.06947041068293412
 -0.0504877330046909 
  0.02030449583778625

In [27]:
absgt(x, y) = abs(y) < abs(x)

absgt (generic function with 1 method)

In [28]:
qsort!(absgt, A)

10-element Array{Float64,1}:
 -1.81397292452982   
  1.7551050989528332 
 -1.0761369195107533 
 -0.8167934875972962 
  0.5849497180762366 
  0.4000188338534922 
 -0.3004077766954796 
  0.06947041068293412
 -0.0504877330046909 
  0.02030449583778625

In [29]:
B = rand(-10:10, 20)

20-element Array{Int64,1}:
   6
   8
  -8
  -4
  -7
  -7
   5
  -3
  -6
  -6
   9
   8
  -5
   3
  -3
  -2
  -5
  -3
  -3
 -10

In [30]:
qsort!(B)

20-element Array{Int64,1}:
 -10
  -8
  -7
  -7
  -6
  -6
  -5
  -5
  -4
  -3
  -3
  -3
  -3
  -2
   3
   5
   6
   8
   8
   9

In [31]:
qsort!((x, y)->abs(y) < abs(x), B)

20-element Array{Int64,1}:
 -10
   9
  -8
   8
   8
  -7
  -7
  -6
  -6
   6
  -5
  -5
   5
  -4
  -3
  -3
  -3
  -3
   3
  -2

In [32]:
qsort!(absgt, B)

20-element Array{Int64,1}:
 -10
   9
  -8
   8
   8
  -7
  -7
  -6
  -6
   6
  -5
  -5
   5
  -4
  -3
  -3
  -3
  -3
   3
  -2

In [41]:
b = rand(10_000_000);

In [42]:
b2 = copy(b);

In [43]:
@time sort!(b);

  0.933793 seconds (71 allocations: 3.344 KiB)


In [44]:
@time qsort!(b2);

  3.742682 seconds (4 allocations: 160 bytes)


## Embedding Julia in C/Fortran

See https://docs.julialang.org/en/latest/manual/embedding/